# Importing required libraries

`sqlalchemy` is a "toolkit" that allows access to SQL databases through python. SQL tables can be queried and directly brought to python objects without the need of an intermediate export-import step (such as creating a csv from MySQL Workbench).

Converting data between incompatible type systems is called Object-relational mapping, so you might see `sqlalchemy` referred to as an ORM tool.

Here we'll see the basic workflow for connecting to a local database and transforming it to a `pandas` dataframe. Explore more features in the docs https://www.sqlalchemy.org/

MySQL has its own ORM tool for interacting with python, called `Connector/Python` (https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html). The advantage of `sqlalchemy` is that it can also work with other DBMS such as Oracle, Postgres, etc.

In [ ]:
#!pip3 install pymsql

In [ ]:
import pandas as pd
import getpass
import sqlalchemy as sa

# To connect and extract tables from SQL database.

In [ ]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'

In [ ]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [ ]:
db_connection = sa.create_engine(connection_string)

In [ ]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

In [ ]:
olist_orders = pd.read_sql_query('SELECT * FROM olist.orders;', db_connection)
olist_orders.head()

In [ ]:
query = 'SELECT * FROM publications.authors;'
publications = pd.read_sql_query(query, db_connection)
publications.head()

# Create a database in Mysql and save a table from Pandas into that database.

In [ ]:
db_connection.execute("create database if not exists testing_123;")

In [ ]:
db_connection.execute("commit")

In [ ]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

In [ ]:
test_table = pd.DataFrame({"a" : [1,2,3], "b" : [4,5,6]})
test_table

In [ ]:
test_table.to_sql(name="test_table", con=db_connection, schema="testing_123")